In [1]:
import pandas as pd

In [2]:
endangeredSpecies = pd.read_csv('endangeredSpecies.csv')
endangeredSpecies.head()

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas
0,3548724,Arthonia obscurella,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Arthoniaceae,Arthonia,obscurella,Aceptado,CR,"[(4.59806, -74.07583)]"
1,3548779,Arthonia septemlocularis,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Arthoniaceae,Arthonia,septemlocularis,Aceptado,CR,"[(4.59806, -74.07583)]"
2,10800176,Ancistrosporella leucophila,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Roccellaceae,Ancistrosporella,leucophila,Aceptado,CR,"[(6.98931, -73.04892), (4.0, -72.0), (6.98701,..."
3,10874088,Byssoloma permutans,Fungi,Ascomycota,Lecanoromycetes,Lecanorales,Byssolomataceae,Byssoloma,permutans,Aceptado,CR,[]
4,5516240,Lecanactis proximans,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Roccellaceae,Lecanactis,proximans,Aceptado,CR,"[(4.0, -72.0), (4.0, -72.0), (4.645293, -74.06..."


In [8]:
endangeredAmphibiaSpecies = endangeredSpecies.loc[endangeredSpecies['clase'] == 'Amphibia']
endangeredAmphibiaSpecies.head()

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas
1526,2431315,Bolitoglossa capitana,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,capitana,Válido,CR,"[(4.53148, -74.336), (4.53148, -74.336), (4.53..."
1527,9338198,Bolitoglossa guaneae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,guaneae,Válido,VU,"[(5.617806, -73.956917), (5.617694, -73.956806..."
1528,2431289,Bolitoglossa hiemalis,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,hiemalis,Válido,VU,"[(4.063167, -76.499361), (4.063167, -76.499361..."
1529,2431296,Bolitoglossa hypacra,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,hypacra,Válido,EN,"[(6.453389, -76.099083), (6.453389, -76.099083..."
1530,9577531,Bolitoglossa leandrae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,leandrae,Válido,CR,"[(4.86, -73.26), (4.86, -73.26), (4.15, -73.65)]"


In [15]:
endangeredAmphibiaSpecies.head().to_csv('endangeredAmphibiaSpecies.csv', index=False)

In [33]:
import pandas as pd
import folium

# Read the CSV data into a pandas DataFrame
df = pd.read_csv('endangeredAmphibiaSpecies.csv')

# Create a base map with dark theme
m = folium.Map(location=[4.5709, -74.2973], zoom_start=8, tiles='CartoDB dark_matter')

# Create a feature group for each conservation status
threatened_fg = folium.FeatureGroup(name='Threatened Species')
endangered_fg = folium.FeatureGroup(name='Endangered Species')
critical_fg = folium.FeatureGroup(name='Critically Endangered Species')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    species = row['nombre_cientifico']
    status = row['estado_de_amenaza']
    coordinates = eval(row['coordenadas'])  # Convert string to list of tuples

    # Create a popup with the species name and conservation status
    popup_text = f"<b>{species}</b><br>Conservation Status: {status}"

    # Add circles to the appropriate feature group based on the conservation status
    if status == 'VU':
        for coord in coordinates:
            folium.Circle(location=coord, radius=2500, popup=popup_text, color='darkblue', fill=True,
                          fill_opacity=0.5).add_to(threatened_fg)
    elif status == 'EN':
        for coord in coordinates:
            folium.Circle(location=coord, radius=2500, popup=popup_text, color='orange', fill=True,
                          fill_opacity=0.5).add_to(endangered_fg)
    elif status == 'CR':
        for coord in coordinates:
            folium.Circle(location=coord, radius=2500, popup=popup_text, color='red', fill=True,
                          fill_opacity=0.5).add_to(critical_fg)

# Add the feature groups to the map
threatened_fg.add_to(m)
endangered_fg.add_to(m)
critical_fg.add_to(m)

# Add a layer control to toggle between feature groups
folium.LayerControl().add_to(m)

# Add a legend with matching colors
legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; background-color: rgba(255, 255, 255, 0.8); padding: 15px; border-radius: 5px; box-shadow: 0 0 10px rgba(0, 0, 0, 0.3); z-index: 9999; font-family: Arial, sans-serif; font-size: 14px;">
    <h3 style="margin-top: 0;">Amphibia Species Endangered</h3>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <i class="fa fa-circle" style="color: red; margin-right: 5px;"></i>
        <span>Critically Endangered</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <i class="fa fa-circle" style="color: orange; margin-right: 5px;"></i>
        <span>Endangered</span>
    </div>
    <div style="display: flex; align-items: center;">
        <i class="fa fa-circle" style="color: darkblue; margin-right: 5px;"></i>
        <span>Threatened</span>
    </div>
</div>
    '''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m

# Save the map as an HTML file
m.save('endangered_amphibia_map.html')

In [30]:
endangeredAmphibiaSpecies.loc[endangeredSpecies['estado_de_amenaza'] == 'CR']

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas
1526,2431315,Bolitoglossa capitana,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,capitana,Válido,CR,"[(4.53148, -74.336), (4.53148, -74.336), (4.53..."
1530,9577531,Bolitoglossa leandrae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,leandrae,Válido,CR,"[(4.86, -73.26), (4.86, -73.26), (4.15, -73.65)]"
1539,2426290,Allobates ranoides,Animalia,Chordata,Amphibia,Anura,Aromobatidae,Allobates,ranoides,Válido,CR,"[(4.773, -73.038002), (4.773, -73.038002), (4...."
1540,5216705,Atelopus angelito,Animalia,Chordata,Amphibia,Anura,Bufonidae,Atelopus,angelito,Válido,CR,"[(5.08444, -75.29357)]"
1541,7785931,Atelopus ardila,Animalia,Chordata,Amphibia,Anura,Bufonidae,Atelopus,ardila,Válido,CR,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1785,5218147,Gastrotheca angustifrons,Animalia,Chordata,Amphibia,Anura,Hemiphractidae,Gastrotheca,angustifrons,Válido,CR,[]
1786,5218196,Gastrotheca antomia,Animalia,Chordata,Amphibia,Anura,Hemiphractidae,Gastrotheca,antomia,Válido,CR,"[(4.75305, -76.2911), (4.75305, -76.2911), (4...."
1801,2428303,Hyloscirtus lynchi,Animalia,Chordata,Amphibia,Anura,Hylidae,Hyloscirtus,lynchi,Válido,CR,"[(7.347478, -72.888798), (7.570184, -72.842487..."
1802,2428289,Hyloscirtus pantostictus,Animalia,Chordata,Amphibia,Anura,Hylidae,Hyloscirtus,pantostictus,Válido,CR,[]
